# pyAnVIL FHIR extract and QA

## overview
![image](https://user-images.githubusercontent.com/47808/102566809-16b1fc00-4095-11eb-8cf8-f78952ba0464.png)


## dependencies

> Ensure latest version of pyAnVIL installed

In [1]:
# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip uninstall  -y pyanvil  
# !{sys.executable} -m pip install   pyAnVIL==0.0.6rc2 --upgrade
# !{sys.executable} -m pip show pyanvil
# >>> 0.0.6rc2


> Ensure PFB extract available
![image](https://user-images.githubusercontent.com/47808/99719432-21ab4980-2a61-11eb-8377-6cbd6ab156ed.png)


In [2]:
import os

AVRO_PATH = "/tmp/export_2020-11-05T23_26_49.avro"

if not os.path.isfile(AVRO_PATH):
    !gsutil cp $WORKSPACE_BUCKET/export_2020-11-05T23_26_49.avro AVRO_PATH

assert os.path.isfile(AVRO_PATH), f"{AVRO_PATH} should exist. Please export PFB from https://gen3.theanvil.io/" 

# extract

> Extract all meta data, write terra, gen3 sqlite databases and dashboard summary

In [6]:
"""Extract all workspaces."""
import os
import logging
import json

from anvil.terra.reconciler import Reconciler, Entities
from anvil.util.reconciler import aggregate, DEFAULT_NAMESPACE
from anvil.transformers.fhir.transformer import FhirTransformer
from anvil.dbgap.api import get_accession, get_study


logging.basicConfig(level=logging.ERROR, format='%(asctime)s %(levelname)-8s %(message)s')
DASHBOARD_OUTPUT_PATH = "/tmp"
TERRA_SUMMARY = f"{DASHBOARD_OUTPUT_PATH}/terra_summary.json"
DASHBOARD_OUTPUT_FILE = f"{DASHBOARD_OUTPUT_PATH}/data_dashboard.json"


def harvest_workspaces(consortiums):
    """Harvest all workspaces, return list of workspace_name. Create detailed sqlite graph and summary dashboard."""
    logging.info("Starting aggregation for all AnVIL workspaces, this will take several minutes.")
    
    with open(DASHBOARD_OUTPUT_FILE, 'w') as outs:
        json.dump({'projects': [v for v in aggregate(namespace=DEFAULT_NAMESPACE,
                                                     user_project=os.environ['GOOGLE_PROJECT'],
                                                     consortium=consortiums,
                                                     avro_path=AVRO_PATH
                                                    )]}, outs)
        
    assert os.path.isfile(DASHBOARD_OUTPUT_FILE), f"{DASHBOARD_OUTPUT_FILE} should exist."
    assert os.path.isfile('/tmp/terra.sqlite'), f"'/tmp/terra.sqlite' should exist."
    
    entities = Entities(path='/tmp/terra.sqlite')
    entities.index()
    return [workspace.name for workspace in entities.get_by_name('workspace')]    


def summarize_workspaces():
    """Aggregate harvested workspaces."""
    entities = Entities(path=f'{DASHBOARD_OUTPUT_PATH}/terra.sqlite') 
    # created sql indices
    entities.index()
    emitter = open(TERRA_SUMMARY, "w")
    for workspace in entities.get_by_name('workspace'):
        for subject in workspace.subjects:
            for sample in subject.samples:
                for property, blob in sample.blobs.items():
                    json.dump(
                        {
                            "workspace_id": workspace.id,
                            "subject_id": subject.id,
                            "sample_id": sample.id,
                            "blob": blob['name'],
                        },
                        emitter,
                        separators=(',', ':')
                    )
                    emitter.write('\n')
    emitter.close()    

def write_fhir(workspace_names):
    """Write all fhir objects."""
    entities = Entities(path=f'{DASHBOARD_OUTPUT_PATH}/terra.sqlite')

    for name in workspace_names:
        emitters = {}
        entity = entities.get(name)
        workspace = entity['vertex']
        logging.info(f"Transforming {name}")
        if 'subject' not in entity['edges']:
            logging.error(f"{name} missing subject edges")
            continue
        workspace._subjects = entity['edges']['subject']
        warned_missing_samples = False
        for subject in workspace.subjects:
            entity = entities.get(subject.id)
            if 'sample' not in entity['edges']:
                if not warned_missing_samples:
                    logging.warning(f"{subject.id} missing sample edges")
                warned_missing_samples = True
                continue
            subject.samples = entity['edges']['sample']
            for sample in subject.samples:
                entity = entities.get(sample.id)
                _blobs = entity['edges'].get('blob', None)
                if _blobs:
                    sample.blobs = {b['property_name']: b for b in _blobs}
        transformer = FhirTransformer(workspace=workspace)
        # namespace = workspace.attributes.workspace.namespace
        reconciler_name = workspace.attributes.reconciler_name
        for item in transformer.transform():
            for entity in item.entity():
                resourceType = entity['resourceType']
                dir_path = f"{DASHBOARD_OUTPUT_PATH}/{reconciler_name}/{name}"
                file_path = f"{dir_path}/{resourceType}.json"
                emitter = emitters.get(file_path, None)
                if emitter is None:
                    os.makedirs(dir_path, exist_ok=True)
                    emitter = open(file_path, "w")
                    logging.info(f"Writing {file_path}")
                    emitters[file_path] = emitter
                json.dump(entity, emitter, separators=(',', ':'))
                emitter.write('\n')
        for stream in emitters.values():
            stream.close()


In [4]:
# import firecloud.api as FAPI
# from attrdict import AttrDict
# import re
# def get_projects(namespaces=None, project_pattern=None):
#     """Filter terra workspaces by namespaces and project_pattern.

#     Args:
#         namespaces ([str]): Optional, list of workspace `namespace` to match ex: 'anvil-datastorage'.
#         project_pattern (str): Optional, regexp to match workspace `name` ex: 'AnVIL_CCDG.*'.

#     Returns:
#         dict: keys ['accessLevel', 'public', 'workspace', 'workspaceSubmissionStats']

#     """
#     workspaces = FAPI.list_workspaces()
#     workspaces = workspaces.json()

#     if namespaces:
#         workspaces = [AttrDict(w) for w in workspaces if w['workspace']['namespace'] in namespaces]

#     if project_pattern:
#         workspaces = [AttrDict(w) for w in workspaces if re.match(project_pattern, w['workspace']['name'], re.IGNORECASE)]

#     # normalize fields
#     for w in workspaces:
#         if 'project_files' not in w.workspace:
#             w.workspace.project_files = []
#     return workspaces

# projects = get_projects('anvil-datastorage', 'AnVIL_CMG_.*') + get_projects('anvil-datastorage', 'AnVIL_CCDG_.*') + get_projects('anvil-datastorage', '^AnVIL_GTEx_V8_hg38$') + get_projects('anvil-datastorage', '^1000G-high-coverage-2019$') 
# len(projects)
# >>> 215

## extract & validate

In [5]:
try:
    os.unlink('/tmp/terra.sqlite')
except Exception as e:
    print(e)
    
consortiums = (
    ('CMG', 'AnVIL_CMG_.*'),
    ('CCDG', 'AnVIL_CCDG_.*'),
    ('GTEx', '^AnVIL_GTEx_V8_hg38$'),
    ('ThousandGenomes', '^1000G-high-coverage-2019$'),
)    
workspace_names = [n for n in harvest_workspaces(consortiums)]
print(workspace_names)

[Errno 2] No such file or directory: '/tmp/terra.sqlite'
['AnVIL_CMG_Broad_Muscle_KNC_WGS', 'AnVIL_CMG_BaylorHopkins_HMB-NPU_WES', 'ANVIL_CMG_Broad_Muscle_Laing_WES', 'AnVIL_CMG_Broad_Orphan_VCGS-White_WES', 'AnVIL_CMG_Broad_Muscle_Myoseq_WES', 'AnVIL_CMG_Broad_Heart_Ware_WES', 'AnVIL_CMG_Broad_Muscle_Beggs_WES', 'AnVIL_CMG_Broad_Blood_Gazda_WGS', 'AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES', 'AnVIL_CMG_Broad_Eye_Pierce_WES', 'AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WGS', 'AnVIL_CMG_Broad_Muscle_Topf_WES', 'AnVIL_CMG_Broad_Blood_Sankaran_WES', 'AnVIL_CMG_Broad_Brain_Walsh_WES', 'AnVIL_CMG_Broad_Muscle_Kang_WES', 'AnVIL_CMG_Broad_Kidney_Hildebrandt_WES', 'AnVIL_CMG_Broad_Muscle_OGrady_WES', 'AnVIL_CMG_Broad_Brain_Gleeson_WES', 'AnVIL_CMG_UWash_GRU', 'AnVIL_CMG_Broad_Brain_Gleeson_WGS', 'AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS', 'AnVIL_CMG_Broad_Blood_Gazda_WES', 'AnVIL_CMG_Broad_Orphan_Manton_WES', 'AnVIL_CMG_Broad_Brain_Engle_WGS', 'AnVIL_CMG_Broad_Heart_PCGC-Tristani_WGS', 'AnVIL_CMG_Broad_

# QA Report 

> Show reconciliation with terra, gen3

##  Issues/Questions arising from Terra

In [8]:
# python json serializer setup

import datetime
import json
import os
from anvil.util.reconciler import flatten
import pandas as pd

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code."""
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError("Type %s not serializable" % type(obj))


# validate output summary and 
assert os.path.isfile(DASHBOARD_OUTPUT_FILE), "dashboard should exist"
with open(DASHBOARD_OUTPUT_FILE, 'r') as inputs:
    dashboard_data = json.load(inputs)
    
# Flatten dashboard into tsv

(flattened, column_names) = flatten(dashboard_data['projects'])
df = pd.DataFrame(flattened)  
df.columns = column_names
# Print the data  (all rows, all columns)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# export create a tsv from dataframe
df.to_csv("/tmp/data_dashboard.tsv", sep="\t")
df

,source,workspace,accession,Bai,Bam,Crai,Cram,Md5,Tbi,Vcf,size,Project,Samples,Subject,dbgap_sample_count_mismatch,inconsistent_entityName,inconsistent_subject,missing_accession,missing_blobs,missing_samples,missing_schema,missing_sequence,missing_subjects
0,CMG,AnVIL_CMG_Broad_Muscle_KNC_WGS,phs001272.v1.p1,,,,,,,,0,1,14,14,,,,,,,,True,
1,CMG,AnVIL_CMG_BaylorHopkins_HMB-NPU_WES,None,,11549187605543,,,77519,,,11549187683062,1,789,789,,,True,,True,,,,
2,CMG,ANVIL_CMG_Broad_Muscle_Laing_WES,phs001272.v1.p1,,,,,,,,0,1,31,31,,,,,,,,True,
3,CMG,AnVIL_CMG_Broad_Orphan_VCGS-White_WES,phs001272.v1.p1,,,,,,,,0,1,447,447,,,,,,,,True,
4,CMG,AnVIL_CMG_Broad_Muscle_Myoseq_WES,phs001272.v1.p1,,,,,,,,0,1,1280,1280,,,,,,,,True,
5,CMG,AnVIL_CMG_Broad_Heart_Ware_WES,phs001272.v1.p1,,,,,,,,0,1,10,10,,,,,,,,True,
6,CMG,AnVIL_CMG_Broad_Muscle_Beggs_WES,phs001272.v1.p1,,,,,,,,0,1,109,109,,,,,,,,True,
7,CMG,AnVIL_CMG_Broad_Blood_Gazda_WGS,phs001272.v1.p1,,,,,,,,0,1,0,0,,,,,True,True,True,,True
8,CMG,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES,phs001272.v1.p1,,,,,,,,0,1,31,31,,,,,,,,True,
9,CMG,AnVIL_CMG_Broad_Eye_Pierce_WES,phs001272.v1.p1,,,,,,,,0,1,602,602,,,,,,,,True,


## summarize terra exceptions

> Extract the list of data transformation problems encountered [see more on dashboard exceptions](https://github.com/anvilproject/client-apis/wiki/dashboard-exceptions)

In [9]:
flattened = []
problems = set([problem for project in dashboard_data['projects'] for problem in project['problems']])
for problem in problems:
    projects = [project['project_id'] for project in dashboard_data['projects'] if problem in project['problems']]
    flattened.append([problem, ','.join(projects)])

# Print the data  (all rows, all columns)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')

df = pd.DataFrame(flattened)  
df.columns = ['problem', 'affected_workspaces']
df = df.style.set_properties(**{'text-align': 'left'})
df 

,problem,affected_workspaces
0,missing_subjects,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_Freeze2_VCFAggregation,AnVIL_CCDG_WashU_CVD_EOCAD_BioImage_WGS,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_Deposit"
1,missing_blobs,"AnVIL_CMG_BaylorHopkins_HMB-NPU_WES,AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WashU_CVD_EOCAD_Harvard-Costa-Rica_WGS,AnVIL_CCDG_Broad_NP_Epilepsy_DEUUPM_HMB_MDS_WES,AnVIL_CCDG_Broad_NP_Epilepsy_USAUPN_GRU_NPU_WES,AnVIL_CCDG_Broad_AI_IBD_McGovern_WGS,AnVIL_ccdg_asc_ndd_daly_talkowski_AGRE-FEMF_asd_exome,AnVIL_CCDG_Broad_NP_Epilepsy_LEBABM_GRU_WES,anvil_ccdg_broad_ai_ibd_daly_mcgovern_share_wes,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS,AnVIL_CCDG_Broad_Deposit"
2,missing_samples,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_ccdg_asc_ndd_daly_talkowski_lattig_asd_exome,AnVIL_CCDG_WashU_CVD_EOCAD_WashU-CAD_GRU-IRB_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_METSIM_WGS,AnVIL_CCDG_NYGC_NP_Autism_ACE2_DS-MDS_WGS,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Broad_AI_IBD_Cho_WGS,AnVIL_CCDG_NYGC_NP_Autism_SAGE_WGS,AnVIL_ccdg_asc_ndd_daly_talkowski_menashe_asd_exome,AnVIL_CCDG_Baylor_CVD_HemStroke_ERICH_WGS,AnVIL_CCDG_Baylor_CVD_EOCAD_SoL_WGS,AnVIL_CCDG_Freeze2_VCFs,AnVIL_CCDG_WashU_CVD_EOCAD_WashU-CAD_DS_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_Emory_WGS,AnVIL_CCDG_NYGC_NP_Autism_ACE2_GRU-MDS_WGS,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_ccdg_asc_ndd_daly_talkowski_brusco_asd_exome,AnVIL_CCDG_Baylor_CVD_ARIC,AnVIL_CCDG_Broad_Deposit,AnVIL_GTEx_V8_hg38"
3,missing_accession,"AnVIL_CCDG_WashU_CVD_EOCAD_BioMe_WGS,AnVIL_CCDG_Broad_CVD_AFib_Penn_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_METSIM_WGS,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_CVD_EOCAD_PROMIS_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_Duke_WGS,AnVIL_CCDG_Broad_CVD_AFib_Duke_WGS"
4,missing_schema,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_Deposit"
5,inconsistent_entityName,"AnVIL_CCDG_WashU_AI_T1D_T1DGC_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_Harvard-Costa-Rica_WGS,AnVIL_CCDG_NYGC_NP_Autism_SSC_WGS,AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS"
6,missing_sequence,"AnVIL_CMG_Broad_Muscle_KNC_WGS,ANVIL_CMG_Broad_Muscle_Laing_WES,AnVIL_CMG_Broad_Orphan_VCGS-White_WES,AnVIL_CMG_Broad_Muscle_Myoseq_WES,AnVIL_CMG_Broad_Heart_Ware_WES,AnVIL_CMG_Broad_Muscle_Beggs_WES,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES,AnVIL_CMG_Broad_Eye_Pierce_WES,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WGS,AnVIL_CMG_Broad_Muscle_Topf_WES,AnVIL_CMG_Broad_Blood_Sankaran_WES,AnVIL_CMG_Broad_Brain_Walsh_WES,AnVIL_CMG_Broad_Muscle_Kang_WES,AnVIL_CMG_Broad_Kidney_Hildebrandt_WES,AnVIL_CMG_Broad_Muscle_OGrady_WES,AnVIL_CMG_Broad_Brain_Gleeson_WES,AnVIL_CMG_Broad_Kidney_Hildebrandt_WGS,AnVIL_CMG_Broad_Orphan_Manton_WES,AnVIL_CMG_Broad_Heart_PCGC-Tristani_WGS,AnVIL_CMG_Broad_Muscle_KNC_WES,AnVIL_CMG_Broad_Heart_Seidman_WES,AnVIL_CMG_Broad_Muscle_Bonnemann_WGS,AnVIL_CMG_Broad_Muscle_Bonnemann_WES,AnVIL_CMG_Broad_Orphan_Manton_WGS,AnVIL_CMG_Broad_Muscle_Ravenscroft_WES,AnVIL_CMG_Broad_Orphan_VCGS-White_WGS,AnVIL_CMG_Broad_Muscle_Myoseq_WGS,AnVIL_CMG_Broad_Muscle_Kang_WGS,AnVIL_CMG_Broad_Eye_Pierce_WGS"
7,dbgap_sample_count_mismatch,"phs001272.v1.p1,phs001489.v1.p1,phs00

## list consistent terra workspaces

In [10]:
# list consistent workspaces

df = pd.DataFrame([project['project_id'] for project in dashboard_data['projects'] if len(project['problems']) == 0])  
df.columns = ['workspace']
df = df.style.set_properties(**{'text-align': 'left'})
df 

,workspace
0,anvil_ccdg_broad_ai_ibd_daly_niddk_cho_wes
1,AnVIL_CCDG_Broad_NP_Epilepsy_ITAUBG_DS_EPI_NPU_MDS_WES
2,AnVIL_CCDG_Broad_NP_Epilepsy_USAMGH_MGBB_HMB_MDS_WES
3,AnVIL_CCDG_Broad_NP_Epilepsy_USACRW_EPI_ASZ_MED_MDS_WES
4,AnVIL_ccdg_asc_ndd_daly_talkowski_chung_asd_exome
5,AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS
6,AnVIL_CCDG_Broad_AI_IBD_Brant_DS-IBD_WGS
7,AnVIL_ccdg_asc_ndd_daly_talkowski_ac-boston_asd_exome
8,AnVIL_CCDG_Broad_NP_Epilepsy_AUSAUS_EPI_BA_ID_MDS_WES
9,anvil_ccdg_broad_ai_ibd_niddk_daly_silverberg_wes


## Issues/Questions arising from Gen3 PFB

In [11]:
# create 
summarize_workspaces()

In [24]:
conn = sqlite3.connect('/tmp/gen3-drs.sqlite')
cur = conn.cursor()

#
# load the terra dashboard summary into db
#
cur.executescript("""
-- 
drop table if exists terra_details ;
CREATE TABLE IF NOT EXISTS terra_details (
    workspace_id text,
    subject_id text,
    sample_id text,
    blob text
);
""")

conn.commit()

with open(f"{DASHBOARD_OUTPUT_PATH}/terra_summary.json", 'rb') as fo:
    for line in fo.readlines():
        record = json.loads(line)
        cur.execute("REPLACE into terra_details values (?, ?, ?, ?);", (record['workspace_id'], record['subject_id'], record['sample_id'], record['blob'],))
conn.commit()

cur.executescript("""
CREATE UNIQUE INDEX IF NOT EXISTS terra_details_idx ON terra_details(workspace_id, subject_id, sample_id, blob);
""")
conn.commit()


#
# reconcile with gen3
#

sql = """

-- missing sequencing
drop table if exists flattened ;
create table flattened
as
select
    json_extract(su.json, '$.object.project_id') as "project_id",
    json_extract(su.json, '$.object.anvil_project_id') as "anvil_project_id",
    su.name as "subject_type",
    su.key as "subject_id",
    json_extract(su.json, '$.object.participant_id') as "participant_id",
    json_extract(su.json, '$.object.submitter_id') as "subject_submitter_id",
    sa.name as "sample_type",
    sa.key  as "sample_id",
    json_extract(sa.json, '$.object.sample_id') as "sample_sample_id",
    json_extract(sa.json, '$.object.submitter_id') as "sample_submitter_id",
    json_extract(sa.json, '$.object.specimen_id') as "sample_specimen_id",
    'sequencing' as "sequencing_type",
    sequencing_edge.src  as "sequencing_id",
    json_extract(sq.json, '$.object.submitter_id') as "sequencing_submitter_id",
    json_extract(sq.json, '$.object.ga4gh_drs_uri') as "ga4gh_drs_uri"
    from vertices as su 
        join edges as sample_edge on sample_edge.dst = su.key and sample_edge.src_name = 'sample'
            join vertices as sa on sample_edge.src = sa.key  
                left join edges as sequencing_edge on sequencing_edge.dst = sa.key and sequencing_edge.src_name = 'sequencing'
                    join vertices as sq on sequencing_edge.src = sq.key 

    where           
    su.name = 'subject'            ;


drop table if exists summary ;
create table summary
as
 select f.project_id, f.anvil_project_id, 
    count(distinct f.subject_id) as "subject_count", 
    count(distinct f.sample_id) as "sample_count",
    count(distinct m.sequencing_id) as "sequencing_count",
    count(distinct m.ga4gh_drs_uri) as "ga4gh_drs_uri_count"
    from flattened as f
        left join flattened as m on f.project_id = m.project_id and f.anvil_project_id = m.anvil_project_id
    group by f.project_id, f.anvil_project_id;


drop table if exists reconcile_counts;
create table reconcile_counts as 
select w.workspace_id,
    count(distinct w.sample_id) as "terra_sample_id_count",
    count(distinct f.sample_submitter_id) as "gen3_sample_id_count",
    count(distinct w.blob) as "terra_blob_count",
    count(distinct f.ga4gh_drs_uri) as "gen3_drs_uri_count"
    from terra_details as w 
        left join flattened as f on (w.sample_id || '_sample' = f.sample_submitter_id)
group by w.workspace_id    
having gen3_sample_id_count > 0 
UNION
select w.workspace_id,
    count(distinct w.sample_id) as "terra_sample_id_count",
    count(distinct f.sample_submitter_id) as "gen3_sample_id_count",
    count(distinct w.blob) as "terra_blob_count",
    count(distinct f.ga4gh_drs_uri) as "gen3_drs_uri_count"
    from terra_details as w 
        left join flattened as f on (w.sample_id   = f.sample_submitter_id)
group by w.workspace_id    
having gen3_sample_id_count > 0 
UNION
select w.workspace_id,
    count(distinct w.sample_id) as "terra_sample_id_count",
    count(distinct f.sample_submitter_id) as "gen3_sample_id_count",
    count(distinct w.blob) as "terra_blob_count",
    count(distinct f.ga4gh_drs_uri) as "gen3_drs_uri_count"
    from terra_details as w 
        left join flattened as f on (w.sample_id   = f.sample_specimen_id)
group by w.workspace_id    
having gen3_sample_id_count > 0 
;

insert into reconcile_counts
select w.workspace_id,
    count(distinct w.sample_id) as "terra_sample_id_count",
    0 as "gen3_sample_id_count",
    count(distinct w.blob) as "terra_blob_count",
    0 as "gen3_drs_uri_count"
from terra_details  as w
where workspace_id not in ( select distinct workspace_id from reconcile_counts ) 
group by w.workspace_id    ;
;

drop table if exists missing_sequencing;

create table missing_sequencing
as 
select s.key, s.submitter_id  from vertices  as s
where s.name = 'sample' 
and
not EXISTS(
    select q.src from edges as q where q.dst = s.key 
) ;

drop table if exists subjects_missing_sequencing;
create table subjects_missing_sequencing
as
select s.key, s.submitter_id  from vertices  as s
where s.name = 'subject' 
and s.key in
(
    select q.dst from edges as q where q.src in (select ms.key from missing_sequencing as ms)
) ;


"""

cur.executescript(sql)
conn.commit()

## PFB contains gen3 projects without anvil(terra) project

In [25]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('/tmp/gen3-drs.sqlite')
cur = conn.cursor()

df = pd.read_sql_query("SELECT * from summary where anvil_project_id is null;", conn)
df

,project_id,anvil_project_id,subject_count,sample_count,sequencing_count,ga4gh_drs_uri_count
0,CCDG-phs001259-DS-CARD-MDS-GSO,None,2158,2159,0,0
1,CCDG-phs001398-GRU,None,496,496,0,0
2,CCDG-phs001487-DS-MULTIPLE_DISEASES-IRB-COL-NPU-RD,None,773,773,0,0
3,CCDG-phs001569-GRU,None,1136,1136,0,0
4,CCDG-phs001642-DS-GID,None,31,31,0,0
5,CCDG-phs001642-DS-IBD,None,199,199,0,0
6,CCDG-phs001642-GRU,None,1351,1351,0,0
7,CCDG-phs001642-HMB,None,1248,1248,0,0
8,CF-GTEx,None,981,47068,0,0
9,open_access-1000Genomes,None,3202,3202,0,0


## Not all terra projects found in Gen3

In [26]:
df = pd.read_sql_query("SELECT * from reconcile_counts where gen3_sample_id_count = 0;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,1000G-high-coverage-2019,3202,0,9600,0
1,AnVIL_CCDG_Baylor_CVD_AFib_BioVU_WGS,1122,0,1122,0
2,AnVIL_CCDG_Baylor_CVD_AFib_Groningen_WGS,639,0,639,0
3,AnVIL_CCDG_Baylor_CVD_EOCAD_BioMe_WGS,1201,0,1201,0
4,AnVIL_CCDG_Broad_AI_IBD_Cho_WGS,344,0,688,0
5,AnVIL_CCDG_Broad_AI_IBD_McCauley_WGS,1096,0,2192,0
6,AnVIL_CCDG_Broad_AI_IBD_McGovern_WGS,1633,0,3266,0
7,AnVIL_CCDG_Broad_CVD_AF_BioVU_HMB_GSO_WES,5031,0,10062,0
8,AnVIL_CCDG_Broad_CVD_AF_ENGAGE_DS_WES,13621,0,27242,0
9,AnVIL_CCDG_Broad_CVD_AFib_AFLMU_WGS,248,0,496,0


## Terra / Gen3 samples count mismatch

In [27]:
df = pd.read_sql_query("SELECT * from reconcile_counts where gen3_sample_id_count > 0 and gen3_sample_id_count <> terra_sample_id_count;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,AnVIL_CCDG_Broad_CVD_EOCAD_VIRGO_WGS,2159,2148,4318,4296


## Terra / Gen3 blob/drs count alignment

In [28]:
df = pd.read_sql_query("SELECT * from reconcile_counts where terra_sample_id_count = gen3_sample_id_count and terra_blob_count = gen3_drs_uri_count;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count
0,AnVIL_CCDG_Broad_AI_IBD_Brant_DS-IBD_WGS,199,199,398,398
1,AnVIL_CCDG_Broad_AI_IBD_Brant_HMB_WGS,904,904,1808,1808
2,AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS,1351,1351,2702,2702
3,AnVIL_CCDG_Broad_AI_IBD_Newberry_WGS,31,31,62,62
4,AnVIL_CCDG_Broad_CVD_EOCAD_PROMIS_WGS,1136,1136,2272,2272
5,AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS,773,773,1546,1546
6,AnVIL_CCDG_Broad_CVD_Stroke_BRAVE_WGS,496,496,992,992


### Terra / Gen3 blob/drs count mismatch

In [29]:

df = pd.read_sql_query("SELECT * from reconcile_counts where terra_sample_id_count = gen3_sample_id_count and terra_blob_count <> gen3_drs_uri_count;", conn)
df

,workspace_id,terra_sample_id_count,gen3_sample_id_count,terra_blob_count,gen3_drs_uri_count


### Unexpected extra files not in terra [leafcutter, bigWig]

In [30]:
pd.set_option('max_colwidth', 256)
df = pd.read_sql_query("select * from terra_details where blob like '%GTEX-12KS4-1526-SM-5EQ6E%' ;", conn)
df

,workspace_id,subject_id,sample_id,blob
0,AnVIL_GTEx_V8_hg38,AnVIL_GTEx_V8_hg38/Su/GTEX-12KS4,GTEx_V8_hg38/Sa/GTEX-12KS4-1526-SM-5EQ6E,gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_RNAseq_BAM_files/GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam
1,AnVIL_GTEx_V8_hg38,AnVIL_GTEx_V8_hg38/Su/GTEX-12KS4,GTEx_V8_hg38/Sa/GTEX-12KS4-1526-SM-5EQ6E,gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_RNAseq_BAM_files/GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam.bai


In [31]:
df = pd.read_sql_query("select key, name, submitter_id  from vertices where submitter_id like '%GTEX-12KS4-1526-SM-5EQ6E%' ;", conn)
df

,key,name,submitter_id
0,0019be63-8121-4869-bc6f-2c2694d6aefd,sample,GTEX-12KS4-1526-SM-5EQ6E
1,4e090504-a493-44ee-a1d2-db8dd34f29d0,sequencing,GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam.bai
2,663f2cb3-467d-46ec-8a05-908cfedc08e1,sequencing,GTEX-12KS4-1526-SM-5EQ6E.leafcutter.junc.gz
3,8abef4da-24c3-4698-9ee6-d2f71f0347ce,sequencing,GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bigWig
4,8bcad8c0-3122-42fd-bdaa-fc912002057a,sequencing,GTEX-12KS4-1526-SM-5EQ6E.Aligned.sortedByCoord.out.patched.md.bam
5,deb14768-af73-44fd-891c-a4206ae94e0e,sequencing,GTEX-12KS4-1526-SM-5EQ6E.SJ.out.tab


## Unexpected Suffixes on gen3 identifiers [_RNASEQ_BAM_FILES, _RNASEQ_BIGWIG]

In [32]:
df = pd.read_sql_query("""select key, name,  json_extract(json, '$.object.submitter_id') as "gen3_submitter_id"   from vertices where gen3_submitter_id like '%_RNASEQ_BAM_FILES' or gen3_submitter_id like '%_BIGWIG'  limit 10;""", conn)
df

,key,name,gen3_submitter_id
0,000b3ad0-94a3-4caa-b1eb-c32d116fe410,sequencing,GTEX-14PJM-0011-R1a-SM-CNPO3.Aligned.sortedByCoord.out.patched.md.bam.bai_RNASEQ_BAM_FILES
1,004af85b-1aee-4287-9b22-9e0e514ff6a4,sequencing,GTEX-11DXW-0626-SM-5N9ER.Aligned.sortedByCoord.out.patched.md.bam_RNASEQ_BAM_FILES
2,00b132b9-8602-41b0-8c4e-9697195795f8,sequencing,GTEX-1A8G7-1126-SM-731ED.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG
3,00b9739d-1e69-418d-b04d-5f1b8664a624,sequencing,GTEX-ZF3C-1426-SM-4WWCD.Aligned.sortedByCoord.out.patched.md.bam_RNASEQ_BAM_FILES
4,00c6695d-6e70-432a-804e-05e4bd7beb59,sequencing,GTEX-1AMEY-1826-SM-72D5L.Aligned.sortedByCoord.out.patched.md.bam.bai_RNASEQ_BAM_FILES
5,00cc7d53-faa1-4d53-93e4-017643865cb2,sequencing,GTEX-OOBK-0126-SM-2YUND.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG
6,00daebe6-352e-4e7f-87ed-c207cc156d20,sequencing,GTEX-13NYS-3126-SM-5KLYV.Aligned.sortedByCoord.out.patched.md.bam_RNASEQ_BAM_FILES
7,00e143a2-6c6e-45c5-975d-8038bc553d15,sequencing,GTEX-14AS3-2226-SM-5S2OX.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG
8,00e4519c-3b46-4fb2-ac42-9f0ca931f7e9,sequencing,GTEX-12696-2826-SM-5FQTY.Aligned.sortedByCoord.out.patched.md.bam.bai_RNASEQ_BAM_FILES
9,010573d8-ce5d-4311-9f9d-46c48c6c561b,sequencing,GTEX-146FH-1226-SM-5NQB6.Aligned.sortedByCoord.out.patched.md.bigWig_RNASEQ_BIGWIG


## subjects without PFB `sequencing` record

In [33]:

df = pd.read_sql_query("select project_id, count(*) from flattened  where subject_id  in (select key from  subjects_missing_sequencing);", conn)
df

,project_id,count(*)
0,CF-GTEx,59316


## unexpected suffix on subject identifiers [_subject]

In [34]:
sql = """select project_id, count( distinct subject_id)   from flattened where subject_submitter_id  like '%_subject'"""
df = pd.read_sql_query(sql, conn)
df

,project_id,count( distinct subject_id)
0,CMG-Broad-GRU,6989


# Transform 
## write FHIR

In [ ]:
write_fhir(workspace_names)

2021-02-12 00:04:57,556 ERROR    AnVIL_CMG_Broad_Blood_Gazda_WGS missing subject edges
2021-02-12 00:05:03,942 ERROR    AnVIL_CMG_Broad_Brain_Gleeson_WGS missing subject edges
2021-02-12 00:05:04,522 ERROR    AnVIL_CMG_Broad_Brain_Engle_WGS missing subject edges
2021-02-12 00:05:04,599 ERROR    AnVIL_CMG_Broad_Kidney_Pollak_WES missing subject edges
2021-02-12 00:05:04,601 ERROR    AnVIL_CMG_Broad_Orphan_Scott_WES missing subject edges
2021-02-12 00:05:05,173 ERROR    AnVIL_CMG_Broad_Blood_Fleming_WES missing subject edges
2021-02-12 00:05:20,544 ERROR    AnVIL_CCDG_Freeze2_VCFAggregation missing subject edges


## write graph of json vertices

In [37]:
from anvil.terra.workspace_graph import WorkspaceGraph

entities = Entities(path='/tmp/terra.sqlite')   
workspace_graph = WorkspaceGraph(path='/tmp/terra-graph.sqlite')

for workspace in entities.get_by_name('workspace'):
    workspace_graph.save(workspace)
workspace_graph.index()    

## "load" 

> Copy results to bucket

In [41]:
# dashboard data
!gsutil cp /tmp/data_dashboard.json  $WORKSPACE_BUCKET
!gsutil cp /tmp/data_dashboard.tsv  $WORKSPACE_BUCKET

# 
!gsutil cp /tmp/terra_summary.json  $WORKSPACE_BUCKET


Copying file:///tmp/data_dashboard.json [Content-Type=application/json]...
/ [1 files][137.9 KiB/137.9 KiB]                                                
Operation completed over 1 objects/137.9 KiB.                                    
Copying file:///tmp/data_dashboard.tsv [Content-Type=text/tab-separated-values]...
/ [1 files][ 25.6 KiB/ 25.6 KiB]                                                
Operation completed over 1 objects/25.6 KiB.                                     
Copying file:///tmp/terra_summary.json [Content-Type=application/json]...
- [1 files][ 93.2 MiB/ 93.2 MiB]                                                
Operation completed over 1 objects/93.2 MiB.                                     


In [40]:
# sqlite databases
!gsutil cp -r /tmp/*.sqlite $WORKSPACE_BUCKET


Copying file:///tmp/gen3-drs.sqlite [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///tmp/pyanvil-cache.sqlite [Content-Type=application/octet-stream]...
Copying file:///tmp/terra-graph.sqlite [Content-Type=application/octet-stream]...
Copying file:///tmp/terra.sqlite [Content-Type=application/octet-stre

In [ ]:
# FHIR data
!gsutil cp -r /tmp/CCDG $WORKSPACE_BUCKET/CCDG
!gsutil cp -r /tmp/CMG $WORKSPACE_BUCKET/CMG
!gsutil cp -r /tmp/GTEx $WORKSPACE_BUCKET/GTEx
!gsutil cp -r /tmp/ThousandGenomes $WORKSPACE_BUCKET/ThousandGenomes

In [29]:
!gsutil ls -r $WORKSPACE_BUCKET

gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/DocumentReference.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Organization.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Patient.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Practitioner.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/ResearchStudy.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/ResearchSubject.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Specimen.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/Task.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/data_dashboard.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/data_dashboard.tsv
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-10-28T16_19_50.avro
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-11-04T17_48_47.avro
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/export_2020-11-05T23_26_49.avro
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/r

## Optional: load bucket contents into FHIR server
![image](https://user-images.githubusercontent.com/47808/102567204-e159de00-4095-11eb-883c-1f36e4790558.png)
![image](https://user-images.githubusercontent.com/47808/102567246-f46cae00-4095-11eb-8090-2fc28f1832e9.png)
